In [4]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv("2017_18_NBA_Stats.csv") # Will want to read in this and 2018/2019 file - figure out weighting between them

file = 'C:\\Users\\stang\\Google Drive\\Data_for_fun\\Fantasy Basketball\\FanDuel\\FanDuel-NBA-2018-10-18-29206-players-list.csv'

### Add any new players

In [6]:
fanduel = pd.read_csv(file)
fanduel["Id"] = fanduel["Id"].str.split('-').str[1]
fanduel["Id"] = fanduel["Id"].astype(int)

fanduel.head()

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier
0,9488,SF,LeBron,LeBron James,James,54.051805,83,11700,LAL@POR,LAL,POR,NaN,NaN,NaN
1,66115,PG,Ben,Ben Simmons,Simmons,64.000000,1,11000,CHI@PHI,PHI,CHI,NaN,NaN,NaN
2,49111,C,Joel,Joel Embiid,Embiid,42.000000,1,10800,CHI@PHI,PHI,CHI,NaN,NaN,NaN
3,20848,PG,Damian,Damian Lillard,Lillard,43.379731,74,9900,LAL@POR,POR,LAL,NaN,NaN,NaN
4,12332,PG,John,John Wall,Wall,41.817073,41,9600,MIA@WAS,WAS,MIA,NaN,NaN,NaN


## Add Positions

In [7]:
# Determine top three positions for each player and add any new players to positions csv.
fd_pos = fanduel
fd_pos = fd_pos.iloc[:, 0:2]
fd_pos.rename(columns={"Id":"ID"}, inplace=True)

pos_all = pd.read_csv("Positions.csv", index_col=0)
pos_all = pd.concat([pos_all,fd_pos], ignore_index=True).drop_duplicates().reset_index(drop=True)
pos_all = pos_all.dropna()

pos_all.head()

,ID,Pos2,Pos3,Position
0,9534,PG,SG,PG
1,41594,C,SF,PF
2,42664,PG,SG,PG
3,87632436,PG,SF,SG
4,19726,PG,SF,SG


In [5]:
pos_all.to_csv("Positions.csv")

In [8]:
conditions = [(pos_all["Position"] == "PF"), (pos_all["Position"] == "C"), (pos_all["Position"] == "SF"),
              (pos_all["Position"] == "SG"), (pos_all["Position"] == "PG")]
choices1 = ["C", "PF", "SG", "PG", "PG"]
pos_all["Pos2"] = np.select(conditions, choices1, default="PG")
choices2 = ["SF", "C", "PF", "SF", "SG"]
pos_all["Pos3"] = np.select(conditions, choices2, default="PG")

data_pos_all = pd.merge(data, pos_all, how='left', left_on='ID', right_on='ID')

data_pos_all["MP"] = np.where(data_pos_all["MP"] == 0, 1, data_pos_all["MP"]) 

data_pos_all["PTS_min"] = data_pos_all["PTS"] / data_pos_all["MP"]
data_pos_all["ORB_min"] = data_pos_all["ORB"] / data_pos_all["MP"]
data_pos_all["DRB_min"] = data_pos_all["DRB"] / data_pos_all["MP"]
data_pos_all["BLK_min"] = data_pos_all["BLK"] / data_pos_all["MP"]
data_pos_all["STL_min"] = data_pos_all["STL"] / data_pos_all["MP"]
data_pos_all["AST_min"] = data_pos_all["AST"] / data_pos_all["MP"]
data_pos_all["TOV_min"] = data_pos_all["TOV"] / data_pos_all["MP"]

data_pos_all.head()

,Rk,ID,Player,Age,Pos,Date,Tm,Home/Away,Opp,Result,...,Pos2,Pos3,Position,PTS_min,ORB_min,DRB_min,BLK_min,STL_min,AST_min,TOV_min
0,1,9772,Ryan Anderson,29-164,F,10/17/2017,HOU,@,GSW,W,...,C,SF,PF,0.393939,0.090909,0.151515,0.030303,0.030303,0.030303,0.000000
1,2,9538,Trevor Ariza,32-109,F,10/17/2017,HOU,@,GSW,W,...,SG,PF,SF,0.210526,0.026316,0.131579,0.000000,0.052632,0.131579,0.026316
2,3,37931,Aron Baynes,30-312,C,10/17/2017,BOS,@,CLE,L,...,PF,C,C,0.315789,0.105263,0.157895,0.052632,0.000000,0.052632,0.105263
3,4,40769,Jordan Bell,22-283,F,10/17/2017,GSW,NaN,HOU,L,...,C,SF,PF,0.666667,0.000000,0.083333,0.000000,0.000000,0.083333,0.083333
4,5,67026,Jaylen Brown,20-358,F-G,10/17/2017,BOS,@,CLE,L,...,SG,PF,SF,0.625000,0.025000,0.125000,0.000000,0.050000,0.000000,0.075000


### Calculate opponent position stats

In [9]:
opp_pts = data_pos_all.groupby(by=["Opp", "Position"], as_index=False) \
    .PTS_min.agg(["mean"]).reset_index()
opp_ORB = data_pos_all.groupby(by=["Opp", "Position"], as_index=False) \
    .ORB_min.agg(["mean"]).reset_index()
    
opp_stats = pd.merge(opp_pts, opp_ORB, how='left', left_on='Opp', right_on='Opp')
opp_stats = opp_stats[opp_stats['Position_x'] == opp_stats['Position_y']]
opp_stats.drop(["Position_y"], axis=1, inplace=True)
opp_stats.rename(columns={"Position_x":"Position","mean_x": "Opp_Pts", "mean_y": "Opp_ORB"}, inplace=True)
opp_stats["Opp_ORB"] = opp_stats["Opp_ORB"] * 1.2

opp_DRB = data_pos_all.groupby(by=["Opp", "Position"], as_index=False) \
    .DRB_min.agg(["mean"]).reset_index()

opp_stats = pd.merge(opp_stats, opp_DRB, how='left', left_on='Opp', right_on='Opp')
opp_stats = opp_stats[opp_stats['Position_x'] == opp_stats['Position_y']]
opp_stats.drop(["Position_y"], axis=1, inplace=True)
opp_stats.rename(columns={"Position_x":"Position","mean": "Opp_DRB"}, inplace=True)
opp_stats["Opp_DRB"] = opp_stats["Opp_DRB"] * 1.2

opp_BLK = data_pos_all.groupby(by=["Opp", "Position"], as_index=False) \
    .BLK_min.agg(["mean"]).reset_index()

opp_stats = pd.merge(opp_stats, opp_BLK, how='left', left_on='Opp', right_on='Opp')
opp_stats = opp_stats[opp_stats['Position_x'] == opp_stats['Position_y']]
opp_stats.drop(["Position_y"], axis=1, inplace=True)
opp_stats.rename(columns={"Position_x":"Position","mean": "Opp_BLK"}, inplace=True)
opp_stats["Opp_BLK"] = opp_stats["Opp_BLK"] * 3

opp_STL = data_pos_all.groupby(by=["Opp", "Position"], as_index=False) \
    .STL_min.agg(["mean"]).reset_index()

opp_stats = pd.merge(opp_stats, opp_STL, how='left', left_on='Opp', right_on='Opp')
opp_stats = opp_stats[opp_stats['Position_x'] == opp_stats['Position_y']]
opp_stats.drop(["Position_y"], axis=1, inplace=True)
opp_stats.rename(columns={"Position_x":"Position","mean": "Opp_STL"}, inplace=True)
opp_stats["Opp_STL"] = opp_stats["Opp_STL"] * 3

opp_AST = data_pos_all.groupby(by=["Opp", "Position"], as_index=False) \
    .AST_min.agg(["mean"]).reset_index()

opp_stats = pd.merge(opp_stats, opp_AST, how='left', left_on='Opp', right_on='Opp')
opp_stats = opp_stats[opp_stats['Position_x'] == opp_stats['Position_y']]
opp_stats.drop(["Position_y"], axis=1, inplace=True)
opp_stats.rename(columns={"Position_x":"Position","mean": "Opp_AST"}, inplace=True)
opp_stats["Opp_AST"] = opp_stats["Opp_AST"] * 1.5

TOV = data_pos_all.groupby(by=["Tm", "Position"], as_index=False) \
    .TOV_min.agg(["mean"]).reset_index()

opp_stats = pd.merge(opp_stats, TOV, how='left', left_on='Opp', right_on='Tm')
opp_stats = opp_stats[opp_stats['Position_x'] == opp_stats['Position_y']]
opp_stats.drop(["Position_y", "Tm"], axis=1, inplace=True)
opp_stats.rename(columns={"Position_x":"Position","mean": "tm_TOV"}, inplace=True)
opp_stats["tm_TOV"] = opp_stats["tm_TOV"] * -1

opp_stats["Opp_Fantasy_Pts"] = opp_stats["Opp_Pts"] + opp_stats["Opp_ORB"] + opp_stats["Opp_DRB"] + \
                                opp_stats["Opp_BLK"] + opp_stats["Opp_STL"] + opp_stats["Opp_AST"] + opp_stats["tm_TOV"]

opp_stats.head()

,Opp,Position,Opp_Pts,Opp_ORB,Opp_DRB,Opp_BLK,Opp_STL,Opp_AST,tm_TOV,Opp_Fantasy_Pts
0,ATL,C,0.398496,0.131563,0.247586,0.129299,0.091977,0.112716,-0.054898,1.056739
6,ATL,PF,0.415290,0.084989,0.213938,0.083753,0.087700,0.114322,-0.042260,0.957733
12,ATL,PG,0.385176,0.027242,0.123094,0.038918,0.121919,0.257698,-0.064281,0.889767
18,ATL,SF,0.421560,0.033774,0.148982,0.047828,0.103312,0.114768,-0.077929,0.792295
24,ATL,SG,0.409860,0.025411,0.128262,0.036274,0.104666,0.140590,-0.059074,0.785989


In [10]:
fanduel.head()

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier
0,9488,SF,LeBron,LeBron James,James,54.051805,83,11700,LAL@POR,LAL,POR,NaN,NaN,NaN
1,66115,PG,Ben,Ben Simmons,Simmons,64.000000,1,11000,CHI@PHI,PHI,CHI,NaN,NaN,NaN
2,49111,C,Joel,Joel Embiid,Embiid,42.000000,1,10800,CHI@PHI,PHI,CHI,NaN,NaN,NaN
3,20848,PG,Damian,Damian Lillard,Lillard,43.379731,74,9900,LAL@POR,POR,LAL,NaN,NaN,NaN
4,12332,PG,John,John Wall,Wall,41.817073,41,9600,MIA@WAS,WAS,MIA,NaN,NaN,NaN


### Join positions to BR file, remove players not playing that night

In [10]:
pos = fanduel.drop(fanduel.columns[[2, 3, 4, 5, 6, 7, 8, 9, 10,11, 12, 13]], axis=1)
conditions = [(pos["Position"] == "PF"), (pos["Position"] == "C"), (pos["Position"] == "SF"),
              (pos["Position"] == "SG"), (pos["Position"] == "PG")]
choices1 = ["C", "PF", "SG", "PG", "PG"]
pos["Pos2"] = np.select(conditions, choices1, default="PG")
choices2 = ["SF", "C", "PF", "SF", "SG"]
pos["Pos3"] = np.select(conditions, choices2, default="PG")

data_pos = pd.merge(data, pos, how='left', left_on='ID', right_on='Id')
data_pos.drop(["Id"], axis=1, inplace=True)
data_pos.dropna(subset=['Position'], inplace=True)

adj_data = pd.merge(data_pos, fanduel, how='right', left_on='ID', right_on='Id')
adj_data.drop(["Salary", "Played", "Game", "Injury Indicator", "Injury Details", "Tier", 
               "FPPG", "Nickname", "First Name", "Last Name", "Id", 
               "Position_y", "Team", "Result", "TRB", "Pos", "Age","FG", "FGA","FG%", "2P", "2PA", "2P%",
              "3P", "3PA", "3P%", "FT", "FTA", "PF", "FT%", "GmSc"], axis=1, inplace=True)
adj_data.rename(columns={"Position_x":"Position", "Opponent":"Tonight_Opp"}, inplace=True)

adj_data["Tonight_Opp"] = np.where(adj_data["Tonight_Opp"] == "NY", "NYK", adj_data["Tonight_Opp"])
adj_data["Tonight_Opp"] = np.where(adj_data["Tonight_Opp"] == "NO", "NOP", adj_data["Tonight_Opp"])

adj_data.head()

,Rk,ID,Player,Date,Tm,Home/Away,Opp,GS,MP,ORB,DRB,AST,STL,BLK,TOV,PTS,Position,Pos2,Pos3,Tonight_Opp
0,13.0,9648.0,Jeff Green,10/17/2017,CLE,NaN,BOS,0.0,14.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,SF,SG,PF,MIA
1,429.0,9648.0,Jeff Green,10/20/2017,CLE,@,MIL,0.0,18.0,1.0,0.0,1.0,1.0,0.0,1.0,10.0,SF,SG,PF,MIA
2,651.0,9648.0,Jeff Green,10/21/2017,CLE,NaN,ORL,0.0,21.0,1.0,0.0,1.0,1.0,0.0,2.0,7.0,SF,SG,PF,MIA
3,1088.0,9648.0,Jeff Green,10/24/2017,CLE,NaN,CHI,0.0,24.0,0.0,2.0,2.0,0.0,0.0,0.0,16.0,SF,SG,PF,MIA
4,1262.0,9648.0,Jeff Green,10/25/2017,CLE,@,BRK,0.0,27.0,0.0,6.0,3.0,1.0,2.0,0.0,18.0,SF,SG,PF,MIA


### Merge stats and opponent stats

In [11]:
# merge on opponent and team for each stat, for Pos1, 2, and 3, and for historical opponent and next opponent

pos_1_prev_opp = pd.merge(adj_data, opp_stats, how='left', left_on=['Opp', 'Position'], right_on=['Opp', 'Position'])
pos_2_prev_opp = pd.merge(adj_data, opp_stats, how='left', left_on=['Opp', 'Pos2'], right_on=['Opp', 'Position'])
pos_3_prev_opp = pd.merge(adj_data, opp_stats, how='left', left_on=['Opp', 'Pos3'], right_on=['Opp', 'Position'])

pos_1_ton_opp = pd.merge(adj_data, opp_stats, how='left', left_on=['Tonight_Opp', 'Position'], right_on=['Opp', 'Position'])
pos_2_ton_opp = pd.merge(adj_data, opp_stats, how='left', left_on=['Tonight_Opp', 'Pos2'], right_on=['Opp', 'Position'])
pos_3_ton_opp = pd.merge(adj_data, opp_stats, how='left', left_on=['Tonight_Opp', 'Pos3'], right_on=['Opp', 'Position'])

pos_1_ton_opp.head()

,Rk,ID,Player,Date,Tm,Home/Away,Opp_x,GS,MP,ORB,...,Tonight_Opp,Opp_y,Opp_Pts,Opp_ORB,Opp_DRB,Opp_BLK,Opp_STL,Opp_AST,tm_TOV,Opp_Fantasy_Pts
0,13.0,9648.0,Jeff Green,10/17/2017,CLE,NaN,BOS,0.0,14.0,0.0,...,MIA,MIA,0.363055,0.032703,0.160721,0.036466,0.093425,0.095191,-0.042779,0.738781
1,429.0,9648.0,Jeff Green,10/20/2017,CLE,@,MIL,0.0,18.0,1.0,...,MIA,MIA,0.363055,0.032703,0.160721,0.036466,0.093425,0.095191,-0.042779,0.738781
2,651.0,9648.0,Jeff Green,10/21/2017,CLE,NaN,ORL,0.0,21.0,1.0,...,MIA,MIA,0.363055,0.032703,0.160721,0.036466,0.093425,0.095191,-0.042779,0.738781
3,1088.0,9648.0,Jeff Green,10/24/2017,CLE,NaN,CHI,0.0,24.0,0.0,...,MIA,MIA,0.363055,0.032703,0.160721,0.036466,0.093425,0.095191,-0.042779,0.738781
4,1262.0,9648.0,Jeff Green,10/25/2017,CLE,@,BRK,0.0,27.0,0.0,...,MIA,MIA,0.363055,0.032703,0.160721,0.036466,0.093425,0.095191,-0.042779,0.738781


### Calculate opponent position adjustment

In [12]:
l = .4
m = (1 - l) / 2

# * team going to play (find relevant position) / team played
adj_data["adj_ORB_Pos1"] = 1.2 * adj_data["ORB"] * pos_1_ton_opp["Opp_ORB"] / pos_1_prev_opp["Opp_ORB"]
adj_data["adj_ORB_Pos2"] = 1.2 * adj_data["ORB"] * pos_2_ton_opp["Opp_ORB"] / pos_2_prev_opp["Opp_ORB"]
adj_data["adj_ORB_Pos3"] = 1.2 * adj_data["ORB"] * pos_3_ton_opp["Opp_ORB"] / pos_3_prev_opp["Opp_ORB"]

adj_data["adj_DRB_Pos1"] = 1.2 * adj_data["DRB"] * pos_1_ton_opp["Opp_DRB"] / pos_1_prev_opp["Opp_DRB"]
adj_data["adj_DRB_Pos2"] = 1.2 * adj_data["DRB"] * pos_2_ton_opp["Opp_DRB"] / pos_2_prev_opp["Opp_DRB"]
adj_data["adj_DRB_Pos3"] = 1.2 * adj_data["DRB"] * pos_3_ton_opp["Opp_DRB"] / pos_3_prev_opp["Opp_DRB"]

adj_data["adj_BLK_Pos1"] = 3 * adj_data["BLK"] * pos_1_ton_opp["Opp_BLK"] / pos_1_prev_opp["Opp_BLK"]
adj_data["adj_BLK_Pos2"] = 3 * adj_data["BLK"] * pos_2_ton_opp["Opp_BLK"] / pos_2_prev_opp["Opp_BLK"] 
adj_data["adj_BLK_Pos3"] = 3 * adj_data["BLK"] * pos_3_ton_opp["Opp_BLK"] / pos_3_prev_opp["Opp_BLK"]

adj_data["adj_STL_Pos1"] = 3 * adj_data["STL"] * pos_1_ton_opp["Opp_STL"] / pos_1_prev_opp["Opp_STL"]
adj_data["adj_STL_Pos2"] = 3 * adj_data["STL"] * pos_2_ton_opp["Opp_STL"] / pos_2_prev_opp["Opp_STL"]
adj_data["adj_STL_Pos3"] = 3 * adj_data["STL"] * pos_3_ton_opp["Opp_STL"] / pos_3_prev_opp["Opp_STL"]

adj_data["adj_AST_Pos1"] = 1.5 * adj_data["AST"] * pos_1_ton_opp["Opp_AST"] / pos_1_prev_opp["Opp_AST"]
adj_data["adj_AST_Pos2"] = 1.5 * adj_data["AST"] * pos_2_ton_opp["Opp_AST"] / pos_2_prev_opp["Opp_AST"]
adj_data["adj_AST_Pos3"] = 1.5 * adj_data["AST"] * pos_3_ton_opp["Opp_AST"] / pos_3_prev_opp["Opp_AST"]

adj_data["adj_PTS_Pos1"] = adj_data["PTS"] * pos_1_ton_opp["Opp_Pts"] / pos_1_prev_opp["Opp_Pts"]
adj_data["adj_PTS_Pos2"] = adj_data["PTS"] * pos_2_ton_opp["Opp_Pts"] / pos_2_prev_opp["Opp_Pts"]
adj_data["adj_PTS_Pos3"] = adj_data["PTS"] * pos_3_ton_opp["Opp_Pts"] / pos_3_prev_opp["Opp_Pts"]

adj_data["adj_TOV_Pos1"] = -1 * adj_data["TOV"] * pos_1_ton_opp["tm_TOV"] / pos_1_prev_opp["tm_TOV"]
adj_data["adj_TOV_Pos2"] = -1 * adj_data["TOV"] * pos_2_ton_opp["tm_TOV"] / pos_2_prev_opp["tm_TOV"]
adj_data["adj_TOV_Pos3"] = -1 * adj_data["TOV"] * pos_3_ton_opp["tm_TOV"] / pos_3_prev_opp["tm_TOV"]

adj_data["exp_ORB"] = l*adj_data["adj_ORB_Pos1"] + m*adj_data["adj_ORB_Pos2"] + m*adj_data["adj_ORB_Pos3"]
adj_data["exp_DRB"] = l*adj_data["adj_DRB_Pos1"] + m*adj_data["adj_DRB_Pos2"] + m*adj_data["adj_DRB_Pos3"]
adj_data["exp_BLK"] = l*adj_data["adj_BLK_Pos1"] + m*adj_data["adj_BLK_Pos2"] + m*adj_data["adj_BLK_Pos3"]
adj_data["exp_STL"] = l*adj_data["adj_STL_Pos1"] + m*adj_data["adj_STL_Pos2"] + m*adj_data["adj_STL_Pos3"]
adj_data["exp_AST"] = l*adj_data["adj_AST_Pos1"] + m*adj_data["adj_AST_Pos2"] + m*adj_data["adj_AST_Pos3"]
adj_data["exp_PTS"] = l*adj_data["adj_PTS_Pos1"] + m*adj_data["adj_PTS_Pos2"] + m*adj_data["adj_PTS_Pos3"]
adj_data["exp_TOV"] = l*adj_data["adj_TOV_Pos1"] + m*adj_data["adj_TOV_Pos2"] + m*adj_data["adj_TOV_Pos3"]

adj_data.dropna(inplace=True)

adj_data["Date"] = pd.to_datetime(adj_data["Date"])
adj_data.sort_values(by="Date", inplace=True)

adj_data["Game_Number"] = adj_data.groupby("ID").cumcount()+1
game_sum = adj_data.groupby("ID").sum()["Game_Number"].reset_index()
game_sum.columns = game_sum.columns.ravel()
exp_pts = pd.merge(adj_data, game_sum, how='left', left_on='ID', right_on='ID')
exp_pts.rename(columns={"Game_Number_x":"Game_Num","Game_Number_y": "Sum_Game_Num"}, inplace=True)

exp_pts["Weight"] = exp_pts["Game_Num"] / exp_pts["Sum_Game_Num"]

exp_pts["Exp_FP"] = exp_pts["Weight"] * (exp_pts["exp_ORB"] + exp_pts["exp_DRB"] + exp_pts["exp_BLK"] +
                                           exp_pts["exp_STL"] + exp_pts["exp_AST"] + exp_pts["exp_PTS"] + 
                                           exp_pts["exp_TOV"])

exp_pts.head()

,Rk,ID,Player,Date,Tm,Home/Away,Opp,GS,MP,ORB,...,exp_DRB,exp_BLK,exp_STL,exp_AST,exp_PTS,exp_TOV,Game_Num,Sum_Game_Num,Weight,Exp_FP
0,105.0,9673.0,Goran Dragic,2017-10-18,MIA,@,ORL,1.0,31.0,3.0,...,2.314526,0.000000,3.224778,8.236520,16.742646,-0.000000,1,666,0.001502,0.050629
1,56.0,9810.0,Jerryd Bayless,2017-10-18,PHI,@,WAS,1.0,28.0,0.0,...,4.162696,0.000000,2.342465,5.185865,13.191073,-1.695048,1,153,0.006536,0.151549
2,256.0,16220.0,Dion Waiters,2017-10-18,MIA,@,ORL,1.0,38.0,0.0,...,0.000000,2.476903,6.194124,5.553231,14.950243,-2.954768,1,136,0.007353,0.192792
3,266.0,58084.0,Justise Winslow,2017-10-18,MIA,@,ORL,0.0,23.0,1.0,...,3.324506,0.000000,2.881428,2.851503,9.880612,-0.992831,1,630,0.001587,0.030313
4,212.0,22541.0,Kelly Olynyk,2017-10-18,MIA,@,ORL,1.0,21.0,0.0,...,5.807333,0.000000,0.000000,4.378181,9.645874,-0.000000,1,741,0.001350,0.026763


### Calculate projection for each player

In [13]:
player_proj = exp_pts.groupby(by=["ID", "Player"], as_index=False).Exp_FP.agg(["sum"]).reset_index()
player_proj.columns = player_proj.columns.ravel()
player_proj.tail(5)

,ID,Player,sum
74,80805.0,Markelle Fultz,13.980458
75,80806.0,Lonzo Ball,33.645196
76,80812.0,Lauri Markkanen,25.937872
77,80815.0,Zach Collins,10.408195
78,80817.0,Bam Adebayo,17.567647


In [14]:
fd = pd.merge(fanduel, player_proj, how='left', left_on='Id', right_on='ID')
fd.dropna(subset=['FPPG', 'sum'], inplace=True)
fd["FPPG"] = fd["sum"].astype(str)
fd["FPPG"] = np.where(fd["Injury Indicator"] == "O", 0, fd["FPPG"]) 

fd.head()

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,ID,Player,sum
0,9488,SF,LeBron,LeBron James,James,52.477849984760375,83,11700,LAL@POR,LAL,POR,NaN,NaN,NaN,9488.0,LeBron James,52.477850
1,66115,PG,Ben,Ben Simmons,Simmons,42.30691736312687,1,11000,CHI@PHI,PHI,CHI,NaN,NaN,NaN,66115.0,Ben Simmons,42.306917
2,49111,C,Joel,Joel Embiid,Embiid,44.93991343814354,1,10800,CHI@PHI,PHI,CHI,NaN,NaN,NaN,49111.0,Joel Embiid,44.939913
3,20848,PG,Damian,Damian Lillard,Lillard,46.88899237428979,74,9900,LAL@POR,POR,LAL,NaN,NaN,NaN,20848.0,Damian Lillard,46.888992
4,12332,PG,John,John Wall,Wall,34.88766793485948,41,9600,MIA@WAS,WAS,MIA,NaN,NaN,NaN,12332.0,John Wall,34.887668


In [17]:
fd.to_csv("lineup.csv")

In [28]:
from pydfs_lineup_optimizer import get_optimizer, Site, Sport
optimizer = get_optimizer(Site.FANDUEL, Sport.BASKETBALL)
optimizer.load_players_from_CSV("lineup.csv")

# Play McGruder
player = optimizer.get_player_by_name('Rodney McGruder') # find player with specified name in your optimizer
optimizer.add_player_to_lineup(player) # lock this player in lineup
OPJ = optimizer.get_player_by_name('Otto Porter Jr.') # find player with specified name in your optimizer
optimizer.add_player_to_lineup(OPJ) # lock this player in lineup

beasley = optimizer.get_player_by_name('Beasley')
beasley.max_exposure = 0.0 # Remove Michael Beasley
jgreen = optimizer.get_player_by_name('Jeff Green')
jgreen.max_exposure = 0.0 # Remove Jeff Green
covington = optimizer.get_player_by_name('Covington')
covington.max_exposure = 0.5 # See lineups without Covington
rivers = optimizer.get_player_by_name('Rivers')
rivers.max_exposure = 0.0 # See lineups without Austin Rivers

In [29]:
k = 7
exposure = k/k
random = False

for lineup in optimizer.optimize(n=k, max_exposure = exposure, randomness=random):
    print(lineup)

 1. PG    Damian Lillard                PG    POR            46.889  9900.0$   
 2. PG    Tyler Johnson                 PG    MIA            23.266  3800.0$   
 3. SG    Kentavious Caldwell-Pope      SG    LAL            27.244  5000.0$   
 4. SG    Rodney McGruder               SG    MIA            13.979  3500.0$   
 5. SF    LeBron James                  SF    LAL            52.478  11700.0$  
 6. SF    Otto Porter Jr.               SF    WAS            27.237  6400.0$   
 7. PF    Kyle Kuzma                    PF    LAL            26.504  6000.0$   
 8. PF    Dario Saric                   PF    PHI            28.768  5900.0$   
 9. C     Jusuf Nurkic                  C     POR            37.216  7800.0$   

Fantasy Points 283.581
Salary 60000.0
 1. PG    Damian Lillard                PG    POR            46.889  9900.0$   
 2. PG    Rajon Rondo                   PG    LAL            24.753  4500.0$   
 3. SG    Kentavious Caldwell-Pope      SG    LAL            27.244  5000.0$   
 